# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [45]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [46]:
%%bigquery --project diesel-dominion-405403
SELECT*
FROM `bigquery-public-data.america_health_rankings.ahr`

Query is running:   0%|          |

Downloading:   0%|          |

,edition,report_type,measure_name,state_name,subpopulation,value,lower_ci,upper_ci,source,source_date
0,2021,2021 Health Disparities,Able-Bodied,Alaska,Other Race,NaN,NaN,NaN,"U.S. Census Bureau, American Community Survey ...",2015-2019
1,2021,2021 Health Disparities,Able-Bodied,Idaho,Other Race,NaN,NaN,NaN,"U.S. Census Bureau, American Community Survey ...",2015-2019
2,2021,2021 Health Disparities,Able-Bodied,Maine,Other Race,NaN,NaN,NaN,"U.S. Census Bureau, American Community Survey ...",2015-2019
3,2021,2021 Health Disparities,Able-Bodied,Montana,Other Race,NaN,NaN,NaN,"U.S. Census Bureau, American Community Survey ...",2015-2019
4,2021,2021 Health Disparities,Able-Bodied,Nebraska,Other Race,NaN,NaN,NaN,"U.S. Census Bureau, American Community Survey ...",2015-2019
...,...,...,...,...,...,...,...,...,...,...
18150,2021,2021 Health Disparities,Uninsured,Tennessee,Hispanic,31.0,30.0,32.0,"U.S. Census Bureau, American Community Survey ...",2015-2019
18151,2021,2021 Health Disparities,Uninsured,Mississippi,Hispanic,32.0,30.0,35.0,"U.S. Census Bureau, American Community Survey ...",2015-2019
18152,2021,2021 Health Disparities,Uninsured,Georgia,Hispanic,33.0,32.0,34.0,"U.S. Census Bureau, American Community Survey ...",2015-2019
18153,2021,2021 Health Disparities,Uninsured,South Dakota,American Indian/Alaska Native,34.0,31.0,36.0,"U.S. Census Bureau, American Community Survey ...",2015-2019


### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [47]:
from google.cloud import bigquery
import pandas as pd

project_id = "diesel-dominion-405403"
client = bigquery.Client(project=project_id)

sql_query = """
SELECT*
FROM `bigquery-public-data.america_health_rankings.ahr`
"""

query_job = client.query(sql_query)
df = query_job.to_dataframe()

def custom_aggregation(series):
    return {
        'mean_edition': series['edition'].mean(),
        'count_unique_measure_name': series['measure_name'].nunique(),
        'max_measure_name_length': series['measure_name'].str.len().max()
    }

aggregated_data = df.groupby(['state_name', 'report_type']).apply(custom_aggregation).reset_index()

print(aggregated_data)

              state_name              report_type  \
0                Alabama  2021 Health Disparities   
1                 Alaska  2021 Health Disparities   
2                Arizona  2021 Health Disparities   
3               Arkansas  2021 Health Disparities   
4             California  2021 Health Disparities   
5               Colorado  2021 Health Disparities   
6            Connecticut  2021 Health Disparities   
7               Delaware  2021 Health Disparities   
8   District of Columbia  2021 Health Disparities   
9                Florida  2021 Health Disparities   
10               Georgia  2021 Health Disparities   
11                Hawaii  2021 Health Disparities   
12                 Idaho  2021 Health Disparities   
13              Illinois  2021 Health Disparities   
14               Indiana  2021 Health Disparities   
15                  Iowa  2021 Health Disparities   
16                Kansas  2021 Health Disparities   
17              Kentucky  2021 Health Disparit

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [48]:
aggregated_data.to_excel('aggregated_data.xlsx')

---

## Submitting Your Work

Submit your work as usual